In [1]:
!pip install keras tensorflow

In [11]:

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from keras.optimizers import Adam
import numpy as np
import random
# Import to_categorical
from tensorflow.keras.utils import to_categorical

# Paragraph
text = """
Technology is evolving at a rapid pace, reshaping how we live, work, and communicate.
Artificial Intelligence, in particular, is transforming industries from healthcare to finance.
As machines learn to analyze data and make decisions, new opportunities and ethical challenges arise.
The integration of smart devices in our daily lives makes convenience the norm, while also raising concerns about privacy.
As we move forward, understanding and guiding technological growth will be key to creating a balanced and inclusive future.
"""

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

# Create input sequences
input_sequences = []
for line in text.split("."):
    tokens = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(tokens)):
        n_gram_sequence = tokens[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Create features and labels
X = input_sequences[:,:-1]
y = input_sequences[:,-1]
y = to_categorical(y, num_classes=total_words) # Now to_categorical is defined
print("--------------------------------------------------------------------")

model = Sequential()
# Change input_dim to total_words or a higher value
model.add(Embedding(input_dim=total_words, output_dim=10, input_length= input_sequences.shape[1]))
model.add(LSTM(50))
model.add(Dense(50, activation='relu'))
model.add(Dense(len(tokenizer.word_index)+1, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

print("--------------------------------------------------------------------------------")


def generate_lines(seed_text, total_lines=3, words_per_line=7):
    generated_text = ""
    current_seed = seed_text

    for _ in range(total_lines):
        line = ""
        for _ in range(words_per_line):
            token_list = tokenizer.texts_to_sequences([current_seed])[0]
            token_list = pad_sequences([token_list], maxlen=max_sequence_len - 1, padding='pre')
            predicted = model.predict(token_list, verbose=0)
            predicted_word_index = np.argmax(predicted, axis=1)[0]

            output_word = ""
            for word, index in tokenizer.word_index.items():
                if index == predicted_word_index:
                    output_word = word
                    break

            # Handle out-of-vocabulary words
            if output_word == "":
                output_word = "[OOV]"  # Replace with a placeholder or handle differently

            current_seed += " " + output_word
            line += output_word + " "
        generated_text += line.strip() + "\n"

    return generated_text

# Try generating 3 lines of text from a seed word
seed = "Technology"
print(generate_lines(seed))

--------------------------------------------------------------------


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

--------------------------------------------------------------------------------
also also opportunities opportunities opportunities opportunities opportunities
opportunities concerns concerns concerns also understanding understanding
evolving opportunities evolving evolving evolving evolving evolving

